In [5]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np


# import useful tools
from glob import glob
from PIL import Image

# import data visualization
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from tqdm.auto import tqdm
import shutil as sh

# import data augmentation
import albumentations as albu

# Setup the paths to train and test images
train=pd.read_csv('dataset/train.csv')
test=pd.read_csv('dataset/test.csv')

Images='dataset/train/'
# Glob the directories and get the lists of train and test images
img = glob(Images + '*')

In [6]:
# !pip install ipywidgets

In [7]:
# Compute at the number of images:
print('Total Number of images is {}'.format(len(img)))

Total Number of images is 10233


In [8]:
print('Number of image in train data are {}'.format(train.shape[0]))
train.head()

Number of image in train data are 10233


,Name,HeadCount
0,1.jpg,11
1,2.jpg,6
2,4.jpg,5
3,5.jpg,9
4,6.jpg,6


In [9]:
# adding width and height in train of images
width, height = [], []
for image in img:
  width.append(Image.open(image).width)
  height.append(Image.open(image).height)
  # train_images["height"] = Image.open(Images+train_images["Name"]).height
print(len(width), len(height))
train["width"] = width
train["height"] = height
train.head()

10233 10233


,Name,HeadCount,width,height
0,1.jpg,11,1080,720
1,2.jpg,6,1080,720
2,4.jpg,5,1080,720
3,5.jpg,9,612,408
4,6.jpg,6,612,408


In [10]:
# whats in bbox.csv
bbox=pd.read_csv('dataset/bbox.csv')
bbox.head()

,Name,xmin,ymin,xmax,ymax,class
0,1.jpg,805,264,917,371,head
1,1.jpg,558,338,584,374,head
2,1.jpg,487,341,515,368,head
3,1.jpg,1014,340,1056,383,head
4,1.jpg,432,364,438,371,head


In [20]:
# Merge all train images with the bounding boxes dataframe
train_images = train.merge(bbox, on='Name', how='left')

In [21]:
df=train_images
df.head()

,Name,HeadCount,width,height,xmin,ymin,xmax,ymax,class
0,1.jpg,11,1080,720,805,264,917,371,head
1,1.jpg,11,1080,720,558,338,584,374,head
2,1.jpg,11,1080,720,487,341,515,368,head
3,1.jpg,11,1080,720,1014,340,1056,383,head
4,1.jpg,11,1080,720,432,364,438,371,head


In [22]:
df['x_center'] = df['xmin'] + df['width']/2
df['y_center'] = df['ymin'] + df['height']/2
df['classes'] = 0

df["x_center"] = df["x_center"]/df["width"]
df["y_center"] = df["y_center"]/df["height"]
df["width"] = df["width"]/max(df["width"])
df["height"] = df["height"]/max(df["height"])

df["x_center"] = df["x_center"]/max(df["x_center"])
df["y_center"] = df["y_center"]/max(df["y_center"])

df['image_id']=df['Name'].str.replace('.jpg','')

df = df[['image_id','xmin', 'ymin', 'width', 'height','x_center','y_center','classes']]

In [23]:
print(f"Shape of df is {df.shape}")
df.head()

Shape of df is (62529, 8)


,image_id,xmin,ymin,width,height,x_center,y_center,classes
0,1,805,264,1.0,1.0,0.356115,0.394899,0
1,1,558,338,1.0,1.0,0.290717,0.441729,0
2,1,487,341,1.0,1.0,0.271918,0.443628,0
3,1,1014,340,1.0,1.0,0.411452,0.442995,0
4,1,432,364,1.0,1.0,0.257356,0.458184,0


In [24]:
df.tail()

,image_id,xmin,ymin,width,height,x_center,y_center,classes
62524,14205,9,52,0.566667,0.566667,0.147181,0.285899,0
62525,14207,160,149,0.566667,0.566667,0.217734,0.394228,0
62526,14207,226,47,0.566667,0.566667,0.248572,0.280315,0
62527,14207,113,41,0.566667,0.566667,0.195774,0.273615,0
62528,14207,170,64,0.566667,0.566667,0.222406,0.299301,0


In [25]:
# df["x_center"] = df["x_center"]/df["width"]
# df["y_center"] = df["y_center"]/df["height"]
# df["width"] = df["width"]/1024
# df["heigth"] = df["height"]/1024

# df["x_center"] = df["x_center"]/max(df["x_center"])
# df["y_center"] = df["y_center"]/max(df["y_center"])
# df.head()

In [27]:
index = list(set(df.image_id))
len(index)

10233

In [29]:
# # code to transform the dataset.

# # ALREADY TRANSFORMED
import os
source = 'dataset'
if True:
    for fold in [0]:
        val_index = index[len(index)*fold//5:len(index)*(fold+1)//5]
        for name,mini in tqdm(df.groupby('image_id')):
            path2save = ''
            if not os.path.exists('convertor/train/labels/'.format(fold)):
                os.makedirs('convertor/train/labels/'.format(fold))
            with open('convertor/train/labels/'.format(fold)+path2save+name+".txt", 'w+') as f:
                row = mini[['classes','x_center','y_center','width','height']].astype(float).values
                row = row.astype(str)
                for j in range(len(row)):
                    text = ' '.join(row[j])
                    f.write(text)
                    f.write("\n")
            if not os.path.exists('convertor/train/images/{}'.format(fold,path2save)):
                os.makedirs('convertor/train/images/{}'.format(fold,path2save))
            sh.copy("C:/Users/prach/Desktop/Prachet/Hackathons/Vista Codefest24/{}/train/{}.jpg".format(source,name),'convertor/train/images/{}.jpg'.format(name))

  0%|          | 0/10233 [00:00<?, ?it/s]

In [30]:
# count
index = list(set(df.image_id))
len(index)

10233

In [31]:
from IPython.display import Image, clear_output

In [32]:
# import required dependencies
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from tqdm.auto import tqdm
import shutil as sh

import matplotlib.pyplot as plt

%matplotlib inline

In [33]:
# !pip install ultralytics

In [35]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.37 🚀 Python-3.11.3 torch-2.2.2+cpu CPU (Intel Core(TM) i5-8265U 1.60GHz)
Setup complete ✅ (8 CPUs, 7.9 GB RAM, 326.9/475.8 GB disk)


In [16]:
# !yolo detect mode=predict model=yolov8n.pt  source="dataset/train/1.jpg"

Ultralytics YOLOv8.1.37 🚀 Python-3.11.3 torch-2.2.2+cpu CPU (Intel Core(TM) i5-8265U 1.60GHz)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

image 1/1 C:\Users\prach\Desktop\Prachet\Hackathons\Vista Codefest24\dataset\train\1.jpg: 448x640 6 persons, 1 tv, 401.6ms
Speed: 23.2ms preprocess, 401.6ms inference, 24.2ms postprocess per image at shape (1, 3, 448, 640)
Results saved to runs\detect\predict2
💡 Learn more at https://docs.ultralytics.com/modes/predict


In [17]:
# from ultralytics import YOLO

# model = YOLO("yolov8n.pt")
# result = model.predict(source="dataset/train/1.jpg")
# print(result)


image 1/1 C:\Users\prach\Desktop\Prachet\Hackathons\Vista Codefest24\dataset\train\1.jpg: 448x640 6 persons, 1 tv, 344.5ms
Speed: 11.6ms preprocess, 344.5ms inference, 6.0ms postprocess per image at shape (1, 3, 448, 640)
[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork',

In [ ]:
from ultralytics import YOLO

# model = YOLO()
model = YOLO("yolov8n.pt")

model.train(data="convertor/data.yaml", epochs=10)

Ultralytics YOLOv8.1.37 🚀 Python-3.11.3 torch-2.2.2+cpu CPU (Intel Core(TM) i5-8265U 1.60GHz)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=convertor/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_

train: Scanning C:\Users\prach\Desktop\Prachet\Hackathons\Vista Codefest24\convertor\train\labels.cache... 10233 images

train: WARNING ⚠️ C:\Users\prach\Desktop\Prachet\Hackathons\Vista Codefest24\convertor\train\images\926.jpg: 1 duplicate labels removed


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning C:\Users\prach\Desktop\Prachet\Hackathons\Vista Codefest24\convertor\train\labels.cache... 10233 images, 

train: WARNING ⚠️ C:\Users\prach\Desktop\Prachet\Hackathons\Vista Codefest24\convertor\train\images\926.jpg: 1 duplicate labels removed


Plotting labels to runs\detect\train\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train
Starting training for 10 epochs...
Closing dataloader mosaic
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/10         0G      2.976      3.492      3.795        101        640:   0%|          | 1/640 [00:33<5:55:23, 3
Exception ignored in: <function TransformNode.set_children.<locals>.<lambda> at 0x000001C8E4C20B80>
Traceback (most recent call last):
  File "c:\Users\prach\Desktop\Prachet\Hackathons\Vista Codefest24\venv\Lib\site-packages\matplotlib\transforms.py", line 198, in <lambda>
    self, lambda _, pop=child._parents.pop, k=id_self: pop(k))
                                                       ^^^^^^
KeyboardInterrupt: 
